In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
# R2 just explains or tells you how much variance is explained by your model
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

## How to split data before training

In [13]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [17]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [18]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
+----------

In [20]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 339|
|   mean|-0.32599387112735057|
| stddev|  10.493359160019095|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                162|
|   mean| 1.4766248285917445|
| stddev|  9.861083162358932|
|    min|-28.046018037776633|
|    max| 22.923352376063196|
+-------+-------------------+



In [22]:
correct_model = lr.fit(train_data)

In [23]:
# Evaluate
test_results = correct_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-25.519848883973413|
|-23.153323106611744|
|-21.913605517367543|
|-20.889083798255296|
| -19.66955115761017|
| -19.33946692543948|
|-15.882097233914529|
|-13.417584394293186|
| -16.70545358231214|
|-17.258130348473987|
| -10.81759002281784|
|-12.998679090171965|
| -17.04458013485516|
|-12.436865893789841|
| -15.75429516635511|
|-12.442284079957366|
| -6.925192736523525|
| -9.208296144783025|
| -7.976661188174212|
| -7.329799220063909|
+-------------------+
only showing top 20 rows



In [25]:
test_results.rootMeanSquaredError

10.073567067470373

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = correct_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.5261691538032216|
|(10,[0,1,2,3,4,5,...|-0.33411701432476837|
|(10,[0,1,2,3,4,5,...|  2.0406144792991383|
|(10,[0,1,2,3,4,5,...|  1.1063210086407578|
|(10,[0,1,2,3,4,5,...|  0.2672151273956168|
|(10,[0,1,2,3,4,5,...|   3.078036649986749|
|(10,[0,1,2,3,4,5,...|-0.26925211736258425|
|(10,[0,1,2,3,4,5,...|  -2.668074646728303|
|(10,[0,1,2,3,4,5,...|  0.9733653100728942|
|(10,[0,1,2,3,4,5,...|  1.5346147354254192|
|(10,[0,1,2,3,4,5,...|   -4.49339056659845|
|(10,[0,1,2,3,4,5,...| -1.7640791627591619|
|(10,[0,1,2,3,4,5,...|  3.2721385731522865|
|(10,[0,1,2,3,4,5,...|  -0.983728882100916|
|(10,[0,1,2,3,4,5,...|  3.2628530888086966|
|(10,[0,1,2,3,4,5,...|    0.58493371452794|
|(10,[0,1,2,3,4,5,...|  -4.715356941365301|
|(10,[0,1,2,3,4,5,...| -1.7376235129999063|
|(10,[0,1,2,3,4,5,...| -2.5124959351986855|
|(10,[0,1,2,3,4,5,...|  -2.56235